In [0]:
!pip install tensorflow
!pip install keras
!pip install gensim
!pip install tqdm

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
!pip install nltk

In [0]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize
import re
from gensim.models import doc2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
import numpy as np
from tqdm import tqdm

In [0]:
df = pd.read_csv('Train.csv')
df.head()

,question,answer_text,distractor
0,Meals can be served,in rooms at 9:00 p. m.,"'outside the room at 3:00 p. m.', 'in the dini..."
1,It can be inferred from the passage that,The local government can deal with the problem...,"'If some tragedies occur again ', ' relevant d..."
2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,"'blame Tommy for his failing grades', 'blame T..."
3,It can be inferred from the passage that,the writer is not very willing to use idioms,'idioms are the most important part in a langu...
4,How can we deal with snake wounds according to...,Stay calm and do n't move .,'Cut the wound and suck the poison out .'


In [0]:
a = list(df['distractor'])
a[0]

"'outside the room at 3:00 p. m.', 'in the dining - room at 6:00 p. m.', 'in the dining - room from 7:30 a. m. to 9:15 p. m.'"

In [0]:
dftp = pd.read_csv('Test.csv')
dftp.head()

,question,answer_text
0,What 'S the main idea of the text ?,The lack of career -- based courses in US high...
1,"In the summer high season , Finland does nt se...",the sun is out at night
2,If you want to apply for Chinese Business Inte...,have to get confirmed at least twice
3,"That afternoon , the boy 's clothes were dry b...",nobody made room for him in the water .
4,Which of the following statements is NOT true ?,There are twelve countries in the World Wildli...


In [0]:
questp = list(dftp['question'])
anstp = list(dftp['answer_text'])
test_tp = questp + anstp

In [0]:
questp = list(df['question'])
anstp = list(df['answer_text'])
test_tp = questp + anstp

In [0]:
ques = list(df['question'])
ans = list(df['answer_text'])
dist = list(df['distractor'])

In [0]:
dist = [i.split(',') for i in tqdm(dist)]

100%|██████████| 31499/31499 [00:00<00:00, 1085599.57it/s]


In [0]:
def clean_text(text):
    text = str(text)
    """
    Function to clean the given text
    :param text: Sentence string
    :return: Cleaned sentence string
    """
    for punct in "/-'":
        text = text.replace(punct, ' ')
    for punct in '&':
        text = text.replace(punct, f' {punct} ')
    for punct in '.!?,#$%\()*+-/:""'';=@""[\\]^_`{|}~':
        text = text.replace(punct, '')
    text = text.lower()
    text = re.sub(r'[?|!|\'|"|#]',r' ',text)
    text = re.sub(r'[.|,|)|(|\|/]',r' ',text)        #Removing Punctuations
    text = re.sub(r"what’s", "what is ", text)
    text = re.sub(r"\’s", " ", text)
    text = re.sub(r"\’ve", " have ", text)
    text = re.sub(r"hasn’t", "has not ", text)
    text = re.sub(r"can’t", "cannot ", text)
    text = re.sub(r"n’t", " not ", text)
    text = re.sub(r"i’m", "i am ", text)
    text = re.sub(r"\’re", " are ", text)
    text = re.sub(r"\’d", " would ", text)
    text = re.sub(r"\’ll", " will ", text)
    text = re.sub(r"\’scuse", " excuse ", text)
    text = re.sub('\d+\d+', 'NUM', text)
    text = text.replace("‘","")
    text = text.replace("”", "")
    text = text.replace("“", "")
    text = text.replace("’",'')
    text = text.replace("‘","")
    text = text.replace('''""''','')
    text = text.replace('''"''','')
    text = re.sub(r' s ', ' ', text)
    return ' '.join(text.split())

In [0]:
!pip install tqdm

In [0]:
def clean(text):
    words = word_tokenize(text)
    clean_words = [re.sub(r'[^A-Za-z]', '', word) for word in words]
    cleaned = ' '.join(clean_words)
    text = cleaned.lower()
    return text
ques = [clean_text(i) for i in tqdm(ques)]
ans = [clean_text(i) for i in tqdm(ans)]
dist = [[clean_text(j) for j in i] for i in tqdm(dist)]

100%|██████████| 31499/31499 [00:01<00:00, 24481.09it/s]


In [0]:
dist[0]

['outside the room at NUM p m',
 'in the dining room at NUM p m',
 'in the dining room from NUM a m to NUM p m']

In [0]:
comb = []
for i, j in tqdm(zip(ques, ans)):
    comb.append(i+' ' +j)
comb[0]

31499it [00:00, 1818182.07it/s]


'meals can be served in rooms at NUM p m'

In [0]:
import pickle
glove = pickle.load(open('glove.42B.300d.pkl','rb')) 

In [0]:
all_data = []
for i in comb:
    all_data.append(i)
for i in dist:
    for j in i:
        all_data.append(j)
all_data = test_tp + all_data

all_data.append('sss')
all_data.append('eee')
print(len(all_data))
all_data[165294]

165295


'eee'

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tk = Tokenizer()
tk.fit_on_texts(pd.Series(all_data))
def pad_seq(comb, max_len):
    X_tokenized= tk.texts_to_sequences(pd.Series(comb).fillna('UNK'))
    X_comb = pad_sequences(X_tokenized, maxlen = max_len, padding = 'post', truncating= 'post')
    return X_comb

Using TensorFlow backend.


In [0]:
tk.word_index['sss']

24505

In [0]:
a = [-1 for i in range(300)]

In [0]:
glove['sss'] = np.ones((1, 300))[0]
glove['eee'] = np.array(a)

In [0]:
import pickle
with open('tk_new.pkl', 'wb') as f:
    pickle.dump(tk, f)

In [0]:
dist_start = []
for i in dist:
    m = []
    for j in i:
        m.append('sss ' + j)
    dist_start.append(m)
dist_start[0]

['sss outside the room at NUM p m',
 'sss in the dining room at NUM p m',
 'sss in the dining room from NUM a m to NUM p m']

In [0]:
dist_end = []
for i in dist:
    m = []
    for j in i:
        m.append(j + ' eee')
    dist_end.append(m)
dist_end[0]

['outside the room at NUM p m eee',
 'in the dining room at NUM p m eee',
 'in the dining room from NUM a m to NUM p m eee']

In [0]:
qa = pad_seq(comb, 20)
dist_seq_start = [[pad_seq(j, 20) for j in i] for i in dist_start]
dist_seq_end = [[pad_seq(j, 20) for j in i] for i in dist_end]

In [0]:
print(len(qa))
print(len(dist_seq_start))

31499
31499


In [0]:
import numpy as np
EMBEDDING_DIM = 300
word_index = tk.word_index
nb_words = len(word_index) + 1
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in glove: 
        embedding_matrix[i] = glove[word]
    elif word.lower() in glove:
        embedding_matrix[i] = glove[word.lower()]

print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
print(nb_words)

Null word embeddings: 1653
24507


In [0]:
x = [i for i in qa]
d1_start = [i[0] for i in dist_seq_start]
d2_start = [i[1] if len(i)>=2 else np.zeros((1, 20)) for i in dist_seq_start]
d3_start = [i[2] if len(i)>=3 else np.zeros((1, 20)) for i in dist_seq_start]

In [0]:
d1_end = [i[0] for i in dist_seq_end]
d2_end = [i[1] if len(i)>=2 else np.zeros((1, 20)) for i in dist_seq_end]
d3_end = [i[2] if len(i)>=3 else np.zeros((1, 20)) for i in dist_seq_end]

In [0]:
print(d3_start[10])
print(d3_end[3])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[   90    17   119   270     2  1680  4486 24506     0     0     0     0
      0     0     0     0     0     0     0     0]]


In [0]:
print(len(dist))
print(len(x))
print(len(d1_start))
print(len(d2_end))
print(len(d3_start))

31499
31499
31499
31499
31499


In [0]:
x = np.array(x)
d1_start = np.array(d1_start)
d2_start = np.array(d1_start)
d3_start = np.array(d1_start)
d3_start.shape

(31499, 1, 20)

In [0]:
d1_end = np.array(d1_end)
d2_end = np.array(d1_end)
d3_end = np.array(d1_end)
d3_end.shape

(31499, 1, 20)

In [0]:
trainx = np.zeros((31499, 20, 300))
for i in range(31499):
    trainx[i] = embedding_matrix[[x[i]]]

traind1s = np.zeros((31499, 20, 300))
for i in range(31499):
    traind1s[i] = embedding_matrix[[d1_start[i]]]

traind2s = np.zeros((31499, 20, 300))
for i in range(31499):
    try:
        traind2s[i] = embedding_matrix[[d2_start[i]]]
    except:
        continue

traind3s = np.zeros((31499, 20, 300))
for i in range(31499):
    try:
        traind3s[i] = embedding_matrix[[d3_start[i]]]
    except:
        continue

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  import sys
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. 

In [0]:
nb_words

24507

In [0]:
inp = 0
model = 0

In [0]:
import keras.utils as ku
traind1e = ku.to_categorical(d1_end[10000:20000], num_classes=24507)
traind2e = ku.to_categorical(d2_end[10000:20000], num_classes=24507)
traind3e = ku.to_categorical(d3_end[10000:20000], num_classes=24507)
traind1e = traind1e.reshape(10000, 20, 24507)
traind2e = traind2e.reshape(10000, 20, 24507)
traind3e = traind3e.reshape(10000, 20, 24507)
traind1e.shape

(10000, 20, 24507)

In [0]:
inp = []
for i, j, k, l in zip(trainx[10000:20000], traind1s[10000:20000], traind2s[10000:20000], traind3s[10000:20000]):
    inp.append([i, j, k, l])
inp = np.array(inp).reshape(10000, 4, 20, 300)
inp.shape

(10000, 4, 20, 300)

In [0]:
import keras
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import LSTM, Dense, Bidirectional, Input, Dropout, Flatten, Embedding, BatchNormalization, Concatenate, concatenate, Add, Lambda

In [0]:
nb_words

24507

In [0]:
def model():
    #inpQA = inp[0]
    inpQA = Input(shape=(4, 20, 300))
    encoder = LSTM(300, return_state=True)
    x1 = Lambda(lambda x:x[:, 0, :])(inpQA)
    print(x1.shape)
    encoder_outputs, state_h, state_c = encoder(x1)
    encoder_states = [state_h, state_c]

    decoder_lstm1 = LSTM(300, return_sequences=True, return_state = True)
    x2 = Lambda(lambda x:x[:, 1, :])(inpQA)
    decoder_outputs1, _, _  = decoder_lstm1(x2,
                                      initial_state=encoder_states)
    decoder_lstm2 = LSTM(300, return_sequences=True, return_state = True)
    x3 = Lambda(lambda x:x[:, 2, :])(inpQA)

    decoder_outputs2,  _, _ = decoder_lstm2(x3,
                                        initial_state=encoder_states)
    decoder_lstm3 = LSTM(300, return_sequences=True, return_state = True)
    x4 = Lambda(lambda x:x[:, 3, :])(inpQA)
    decoder_outputs3, _, _  = decoder_lstm3(x4,
                                        initial_state=encoder_states)

    o1 = Dense(24507, activation = 'softmax')(decoder_outputs1)
    o2 = Dense(24507, activation = 'softmax')(decoder_outputs2)
    o3 = Dense(24507, activation = 'softmax')(decoder_outputs3)
                                        
    cat_conv = [o1, o2, o3]
    

    model = Model(inputs=inpQA, outputs = cat_conv)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
model = model()

(None, 20, 300)


In [0]:
from keras.callbacks import ModelCheckpoint

filepath="/content/drive/My Drive/ML/Valuelabs/temp.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

In [0]:
model.fit(inp, [traind1e, traind2e, traind3e], batch_size=64, epochs=15, verbose=1, callbacks=callbacks)

Epoch 1/15
10000/10000 [==============================] - 267s 27ms/step - loss: 6.2034 - dense_4_loss: 2.0679 - dense_5_loss: 2.0665 - dense_6_loss: 2.0657 - dense_4_accuracy: 0.6976 - dense_5_accuracy: 0.6978 - dense_6_accuracy: 0.6983
Epoch 2/15


/home/ubuntu/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


10000/10000 [==============================] - 258s 26ms/step - loss: 5.4855 - dense_4_loss: 1.8261 - dense_5_loss: 1.8276 - dense_6_loss: 1.8275 - dense_4_accuracy: 0.7087 - dense_5_accuracy: 0.7077 - dense_6_accuracy: 0.7084
Epoch 3/15
10000/10000 [==============================] - 258s 26ms/step - loss: 4.9777 - dense_4_loss: 1.6558 - dense_5_loss: 1.6615 - dense_6_loss: 1.6619 - dense_4_accuracy: 0.7167 - dense_5_accuracy: 0.7162 - dense_6_accuracy: 0.7164
Epoch 4/15
10000/10000 [==============================] - 258s 26ms/step - loss: 4.5051 - dense_4_loss: 1.4972 - dense_5_loss: 1.5050 - dense_6_loss: 1.5049 - dense_4_accuracy: 0.7259 - dense_5_accuracy: 0.7253 - dense_6_accuracy: 0.7255
Epoch 5/15
10000/10000 [==============================] - 258s 26ms/step - loss: 4.0545 - dense_4_loss: 1.3444 - dense_5_loss: 1.3538 - dense_6_loss: 1.3547 - dense_4_accuracy: 0.7383 - dense_5_accuracy: 0.7372 - dense_6_accuracy: 0.7366
Epoch 6/15
10000/10000 [==============================] - 2

In [0]:
model.save('VL2.h5')

In [0]:
model.load_weights('VL2.h5')

In [0]:
def get_seq(sentence, count):
    seqq = tk.texts_to_sequences(pd.Series(sentence).fillna('UNK'))
    seq_vec = pad_sequences(seqq, maxlen = 20, padding = 'post', truncating= 'post')
    emb_vec_qa = embedding_matrix[[seq_vec]]
    emb_vec_qa = emb_vec_qa.reshape(20, 300)
    d1 = [0 for i in range(20)]
    d1 = np.array(d1)
    d1[0] = tk.word_index['sss']
    d1 = embedding_matrix[[d1]]
    d1 = d1.reshape(20, 300)
    d2 = [0 for i in range(20)]
    d2 = np.array(d2)
    d2[0] = tk.word_index['sss']
    d2 = embedding_matrix[[d2]]
    d2 = d2.reshape(20, 300)
    d3 = [0 for i in range(20)]
    d3 = np.array(d3)
    d3[0] = tk.word_index['sss']
    d3 = embedding_matrix[[d3]]
    d3 = d3.reshape(20, 300)
    inn = []
    inn.append([emb_vec_qa, d1, d2, d3])
    inn = np.array(inn)
    inn = inn.reshape(1, 4, 20, 300)
    flag = True
    predict1 = []
    predict2 = []
    predict3 = []
    c = 0
    while flag:
        d1p = np.argmax(model.predict(inn)[0][0][0])
        mydict = tk.word_index
        if d1p==0:
            pred1 = ''
            break
        word = list(mydict.keys())[list(mydict.values()).index(d1p)]

        d1 = [0 for i in range(20)]
        d1 = np.array(d1)
        d1[0] = tk.word_index[word]
        d1 = embedding_matrix[[d1]]
        d1 = d1.reshape(20, 300)
        inn = []
        inn.append([emb_vec_qa, d1, d2, d3])
        inn = np.array(inn)
        inn = inn.reshape(1, 4, 20, 300)
        out = []
        c+=1
        if word == 'eee' or c > count:
            flag = False
            pred1 = ' '.join(predict1)
        predict1.append(word)



#2nd one
    c = 0
    flag = True
    while flag:
        d2p = np.argmax(model.predict(inn)[1][0][0])
        mydict = tk.word_index
        if d2p==0:
            pred2 =  ''
            break
        word = list(mydict.keys())[list(mydict.values()).index(d2p)]

        d2 = [0 for i in range(20)]
        d2 = np.array(d2)
        d2[0] = tk.word_index[word]
        d2 = embedding_matrix[[d2]]
        d2 = d2.reshape(20, 300)
        inn = []
        inn.append([emb_vec_qa, d1, d2, d3])
        inn = np.array(inn)
        inn = inn.reshape(1, 4, 20, 300)
        out = []
        c+=1
        if word == 'eee' or c > count:
            flag = False
            pred2 =' '.join(predict2)          
        predict2.append(word)

#third=================
    c = 0
    flag = True
    while flag:
        d3p = np.argmax(model.predict(inn)[2][0][0])
        mydict = tk.word_index
        if d3p==0:
            pred3 =  ''
            break
        word = list(mydict.keys())[list(mydict.values()).index(d3p)]

        d3 = [0 for i in range(20)]
        d3 = np.array(d3)
        d3[0] = tk.word_index[word]
        d3 = embedding_matrix[[d3]]
        d3 = d3.reshape(20, 300)
        inn = []
        inn.append([emb_vec_qa, d1, d2, d3])
        inn = np.array(inn)
        inn = inn.reshape(1, 4, 20, 300)
        out = []
        c+=1
        if word == 'eee' or c > count:
            flag = False
            pred3 = ' '.join(predict3)
        predict3.append(word)

    return f"'{pred1}', '{pred2}', '{pred3}'"

In [0]:
dftp = pd.read_csv('Test.csv')
dftp.head()

,question,answer_text
0,What 'S the main idea of the text ?,The lack of career -- based courses in US high...
1,"In the summer high season , Finland does nt se...",the sun is out at night
2,If you want to apply for Chinese Business Inte...,have to get confirmed at least twice
3,"That afternoon , the boy 's clothes were dry b...",nobody made room for him in the water .
4,Which of the following statements is NOT true ?,There are twelve countries in the World Wildli...


In [0]:
q = list(dftp['question'])
an = list(dftp['answer_text'])

In [0]:
comb1 = []
for i, j in tqdm(zip(q, an)):
    comb1.append(i+' ' +j)
comb1 = [clean_text(i) for i in tqdm(comb1)]
comb1[0]

13500it [00:00, 1690797.10it/s]
100%|██████████| 13500/13500 [00:00<00:00, 48439.78it/s]


'what the main idea of the text the lack of career based courses in us high schools'

In [0]:
dis1 = [get_seq(i, 10) for i in tqdm(comb1)]
dis1[3]

  0%|          | 0/13500 [00:00<?, ?it/s]/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  after removing the cwd from sys.path.
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if __name__ == '__main__':
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead 

"'were the man could he wanted his man could he', 'the woman did n t the woman did n t', 'their knew were too was too was too was too'"

In [0]:
df_new = pd.read_csv('Results.csv')
df_new['distractor'] = dis1
df_new.to_csv('Results.csv')

In [0]:
l = list(df['distractor'])

In [0]:
str(dist[3])

"['idioms are the most important part in a language', 'nonnative speakers should learn more idioms', 'there are no ways to master idioms']"

In [0]:
l[0]

"'outside the room at 3:00 p. m.', 'in the dining - room at 6:00 p. m.', 'in the dining - room from 7:30 a. m. to 9:15 p. m.'"

In [0]:
get_seq('One part of the homeless population is difficult to estimate . The reason might well be some homeless children are deserted by their families', 10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr

['the homeless children in the homeless children in the homeless children in',
 'the people will be important in two men have two men have',
 'the children can be too much is too much is too much']